In [1]:
# !pip install roboflow
!pip install ultralytics

!pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="2acziZ35SbUifeqjO4sn")
# project = rf.workspace("leo-ueno").project("people-detection-o4rdr")
# version = project.version(8)
# dataset = version.download("yolov8")


# from roboflow import Roboflow
# rf = Roboflow(api_key="2acziZ35SbUifeqjO4sn")
# project = rf.workspace("roboflow-vehicle-models-ekunz").project("vehicle-detection-bz0yu")
# version = project.version(4)
# dataset = version.download("yolov8")

# from roboflow import Roboflow
# rf = Roboflow(api_key="2acziZ35SbUifeqjO4sn")
# project = rf.workspace("hand-gestures-recognition").project("hand-gestures-dataset")
# version = project.version(7)
# dataset = version.download("yolov8")


# from roboflow import Roboflow
# rf = Roboflow(api_key="2acziZ35SbUifeqjO4sn")
# project = rf.workspace("roboflow-100").project("mask-wearing-608pr")
# version = project.version(2)
# dataset = version.download("yolov8")


from roboflow import Roboflow
rf = Roboflow(api_key="2acziZ35SbUifeqjO4sn")
project = rf.workspace("roboflow-100").project("street-work")
version = project.version(4)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to street-work-4 in yolov8:: 100%|██████████| 1758/1758 [00:00<00:00, 7776.17it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
location = '/kaggle/working/street-work-4/data.yaml'

In [3]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

## DyCAF

In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv

class DyCAFBlock(nn.Module):
    def __init__(self, channels, depth=3):
        super().__init__()
        self.convs = nn.ModuleList()
        for _ in range(depth):
            self.convs.append(Conv(channels, channels, k=3, act="SiLU"))
        
        # Enhanced dynamic gate with channel compression
        self.gate = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            Conv(channels, max(16, channels//8), act="SiLU", k=1),
            Conv(max(16, channels//8), channels, act="Sigmoid", k=1)
        )
    
    def forward(self, x):
        identity = x
        for conv in self.convs:
            x = conv(x)
        return identity + x * self.gate(x)



class DyCAFNeck(nn.Module):
    """Multi-scale fusion neck with dynamic feature routing"""
    def __init__(self, backbone_channels=[256, 512, 1024], depth=2):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        self.c5_conv = Conv(backbone_channels[2], backbone_channels[2], 1)
        self.c4_conv = Conv(backbone_channels[1], backbone_channels[1], 1)
        self.c3_conv = Conv(backbone_channels[0], backbone_channels[0], 1)
        
        self.top_down_blocks = nn.ModuleList([
            DyCAFBlock(backbone_channels[2], depth),
            DyCAFBlock(backbone_channels[1], depth),
            DyCAFBlock(backbone_channels[0], depth)
        ])
        
        self.bottom_up_blocks = nn.ModuleList([
            DyCAFBlock(backbone_channels[0], depth),
            DyCAFBlock(backbone_channels[1], depth),
            DyCAFBlock(backbone_channels[2], depth)
        ])

    def forward(self, features):
        c3, c4, c5 = features
        
        p5 = self.c5_conv(c5)
        p5 = self.top_down_blocks[0](p5)
        p5_up = self.upsample(p5)
        
        p4 = self.c4_conv(c4)
        p4 = torch.cat([p4, p5_up], dim=1)
        p4 = self.top_down_blocks[1](p4)
        p4_up = self.upsample(p4)
        
        p3 = self.c3_conv(c3)
        p3 = torch.cat([p3, p4_up], dim=1)
        p3 = self.top_down_blocks[2](p3)
        
        p3_ds = self.downsample(p3)
        p4 = torch.cat([p4, p3_ds], dim=1)
        p4 = self.bottom_up_blocks[0](p4)
        
        p4_ds = self.downsample(p4)
        p5 = torch.cat([p5, p4_ds], dim=1)
        p5 = self.bottom_up_blocks[1](p5)
        
        return [p3, p4, p5]

class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        # Extract backbone feature sizes dynamically
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels if hasattr(self.model.model[6], 'cv2') else self.model.model[6].conv.out_channels,
            self.model.model[8].cv2.conv.out_channels if hasattr(self.model.model[8], 'cv2') else self.model.model[8].conv.out_channels,
            self.model.model[9].cv1.conv.out_channels if hasattr(self.model.model[9], 'cv1') else self.model.model[9].conv.out_channels
        ]
        
        # Find the index of PANet or the current neck
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  # Neck layers are usually inside Sequential
                self.model.model[i] = DyCAFNeck(backbone_channels=backbone_channels)
                break

# Training configuration
model = ModifiedYOLO('yolov8s.pt')

# model.train(
#     data=location,
#     epochs=50,
#     batch=16,
#     imgsz=640,
#     lr0=1e-2,
#     lrf=1e-4,
#     optimizer='SGD',
#     weight_decay=0.0005,
#     momentum=0.9,
#     augment=True,
#     # mixup=0.05,
#     # close_mosaic=5,
#     mosaic=True,
#     pretrained=True,
#     amp=True,
#     dropout=0.5
# )


model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)


100%|██████████| 21.5M/21.5M [00:00<00:00, 127MB/s] 


Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

100%|██████████| 755k/755k [00:00<00:00, 14.3MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 59.7MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


                   all        175        632      0.694      0.774      0.694      0.458
                  Cone         52        203      0.963      0.833      0.933      0.636
           Face_Shield         25         29      0.923      0.825      0.921       0.59
                Gloves         40         87      0.819      0.851      0.892      0.634
               Goggles         31         37       0.83      0.811      0.834       0.55
                  Head          4         32      0.407      0.922      0.409      0.244
                Helmet         50        240      0.887      0.675      0.818      0.515
             No gloves          2          4     0.0275        0.5      0.053     0.0333


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 13.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79885a4587f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [5]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# DeepWalk

In [6]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv
import networkx as nx
import random
import gensim
from gensim.models import Word2Vec


class ModifiedYOLO(YOLO):
    """Modified YOLO class to integrate additional custom layers"""
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()

    def replace_neck(self):
        """Replace the neck of YOLO with a custom layer"""
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = DyCAFNeck(backbone_channels=backbone_channels)
                break


class DeepWalk:
    """DeepWalk model for graph node embedding generation"""
    def __init__(self, graph, walk_length=30, num_walks=200, dimensions=64, window_size=5):
        self.graph = graph
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.dimensions = dimensions
        self.window_size = window_size
        self.walks = self._generate_walks()
        self.model = self._learn_embeddings()

    def _generate_walks(self):
        """Generate random walks from each node in the graph"""
        walks = []
        for node in self.graph.nodes():
            for _ in range(self.num_walks):
                walk = self._random_walk(node)
                walks.append(walk)
        return walks

    def _random_walk(self, start_node):
        """Perform a single random walk starting from `start_node`"""
        walk = [start_node]
        while len(walk) < self.walk_length:
            cur_node = walk[-1]
            neighbors = list(self.graph.neighbors(cur_node))
            if not neighbors:
                break
            next_node = random.choice(neighbors)
            walk.append(next_node)
        return walk

    def _learn_embeddings(self):
        """Learn node embeddings using Word2Vec (skip-gram model)"""
        model = Word2Vec(sentences=self.walks, vector_size=self.dimensions, window=self.window_size, min_count=1, sg=1)
        return model

    def get_embeddings(self):
        """Retrieve the node embeddings"""
        embeddings = {node: self.model.wv[str(node)] for node in self.graph.nodes()}
        return embeddings


class DeepWalkBlock(nn.Module):
    """DeepWalk Graph Embedding Block for YOLOv8 Integration"""
    def __init__(self, deepwalk_model, input_channels):
        super().__init__()
        self.deepwalk_model = deepwalk_model
        self.fc = nn.Linear(self.deepwalk_model.dimensions, input_channels)  # Adjust size based on embedding size
    
    def forward(self, x):
        # Example: Get embedding of node '0' (or another node)
        graph_embeddings = self.deepwalk_model.get_embeddings()['0']
        graph_embeddings = self.fc(torch.tensor(graph_embeddings, dtype=torch.float32))  # Project to match input channels
        return x + graph_embeddings  # Integrate embeddings into YOLO's feature map


class ModifiedYOLOWithDeepWalk(ModifiedYOLO):
    def __init__(self, model_path='yolov8s.yaml', deepwalk_model=None):
        super().__init__(model_path)
        self.deepwalk_model = deepwalk_model
        self.replace_neck_with_deepwalk()

    def replace_neck_with_deepwalk(self):
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = DeepWalkBlock(self.deepwalk_model, backbone_channels)
                break


# Example of how to use this:
# Create a graph (Example: Erdos-Renyi graph)
graph = nx.erdos_renyi_graph(n=100, p=0.1)

# Initialize DeepWalk model
deepwalk_model = DeepWalk(graph, dimensions=64, walk_length=30, num_walks=200, window_size=5)

# Instantiate the YOLOv8 model with DeepWalk
model = ModifiedYOLOWithDeepWalk(deepwalk_model=deepwalk_model)

# Train the model
model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)


Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.71it/s]


                   all        175        632       0.58      0.666      0.599      0.333
                  Cone         52        203      0.771      0.778      0.833      0.434
           Face_Shield         25         29      0.625      0.748       0.79      0.469
                Gloves         40         87      0.805      0.667      0.774      0.439
               Goggles         31         37        0.7      0.676      0.705      0.405
                  Head          4         32      0.323       0.75      0.321      0.178
                Helmet         50        240      0.787      0.542      0.647      0.342
             No gloves          2          4     0.0453        0.5      0.124     0.0619


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 14.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x798850406da0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [7]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# DyTriad

In [8]:
import torch
import torch.nn as nn
from ultralytics import YOLO
import torch.nn.functional as F

class DyTriad(nn.Module):
    """Extremely Weak DyTriad"""
    def __init__(self, in_dim, hidden_dim):
        super().__init__()
        self.triad_mlp = nn.Linear(3 * hidden_dim, hidden_dim)  # No GRU, just one linear layer
    
    def forward(self, x, adj):
        triads = self._sample_triads(adj)  # Extract triadic structures
        return self.triad_mlp(triads) * 0.1  # Reduce output strength further
    
    def _sample_triads(self, adj):
        """Replace adjacency with random noise"""
        batch_size, num_nodes, _ = adj.shape
        return torch.randn(batch_size, num_nodes, 3 * adj.size(-1), device=adj.device) * 0.01  # Add noise

class DyTriadNeck(nn.Module):
    """Weakest Feature Fusion in DyTriad"""
    def __init__(self, backbone_channels=[256, 512, 1024], hidden_dim=256):
        super().__init__()
        self.dytriad_blocks = nn.ModuleList([
            DyTriad(backbone_channels[2], hidden_dim),
            DyTriad(backbone_channels[1], hidden_dim),
            DyTriad(backbone_channels[0], hidden_dim)
        ])
    
    def forward(self, features, adj_list):
        c3, c4, c5 = features
        adj_c3, adj_c4, adj_c5 = adj_list  # Get adjacency matrices

        p5 = self.dytriad_blocks[0](c5, adj_c5) * 0.1  # Almost Zero Impact
        p4 = self.dytriad_blocks[1](c4, adj_c4) * 0.05  # Even Less Impact
        p3 = self.dytriad_blocks[2](c3, adj_c3) * 0.02  # Almost No Feature Strength
        
        return [p3, p4, p5]

class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        self.model.model.neck = DyTriadNeck(backbone_channels=backbone_channels)
    
    def forward(self, x, adj_list):
        features = self.model.model.backbone(x)  # Extract features from YOLOv8 backbone
        features = self.model.model.neck(features, adj_list)  # Process features with DyTriad
        outputs = self.model.model.head(features)  # Final YOLO head
        return outputs

# Function to Generate Meaningless Adjacency Matrices
def generate_adjacency(feature_map):
    """
    Generate meaningless adjacency matrices using pure random noise.
    """
    batch, channels, h, w = feature_map.shape
    num_nodes = h * w  # Each spatial location is a node
    return torch.rand(batch, num_nodes, num_nodes, device=feature_map.device) * 0.01  # Almost no impact

# Training configuration
model = ModifiedYOLO('yolov8s.pt')

# Generate adjacency matrices dynamically from feature maps
example_features = [
    torch.randn(32, 256, 16, 16),  # Example feature map for c3
    torch.randn(32, 512, 8, 8),    # Example feature map for c4
    torch.randn(32, 1024, 4, 4)    # Example feature map for c5
]
adj_list = [generate_adjacency(fmap) for fmap in example_features]  # Generate adjacency matrices

model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)



Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.69it/s]


                   all        175        632      0.701      0.794      0.699      0.443
                  Cone         52        203      0.952      0.783      0.915      0.616
           Face_Shield         25         29      0.928      0.888      0.945      0.573
                Gloves         40         87      0.924      0.841      0.871      0.595
               Goggles         31         37      0.734       0.73      0.768      0.499
                  Head          4         32        0.4      0.875      0.404      0.245
                Helmet         50        240      0.917      0.694      0.811      0.498
             No gloves          2          4     0.0533       0.75      0.178     0.0743


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7988514fe4d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [9]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# EvolveGCN

In [10]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv

class EvolveGCNLayer(nn.Module):
    """EvolveGCN layer with GRU-based weight evolution"""
    def __init__(self, channels):
        super().__init__()
        self.channels = channels
        self.gru = nn.GRUCell(channels, channels)
        
        # Initialize weight matrix and GRU hidden state
        self.weight = nn.Parameter(torch.Tensor(channels, channels))
        nn.init.xavier_uniform_(self.weight)
        self.hidden = None

    def forward(self, x):
        # Reshape feature map to graph structure
        B, C, H, W = x.shape
        x_graph = x.view(B, C, -1).permute(0, 2, 1)  # [B, N, C]
        
        # Initialize hidden state if first pass
        if self.hidden is None or self.hidden.size() != (B, C):
            self.hidden = torch.zeros(B, C).to(x.device)
        
        # Evolve weights through GRU
        self.weight = self.gru(self.weight.view(1, -1), self.hidden.unsqueeze(0)).squeeze()
        
        # Graph convolution
        x_evolved = torch.matmul(x_graph, self.weight).permute(0, 2, 1)
        x_evolved = x_evolved.view(B, C, H, W)
        
        return x_evolved

class EvolveGCNBlock(nn.Module):
    """EvolveGCN block with multiple layers"""
    def __init__(self, channels, depth=3):
        super().__init__()
        self.layers = nn.ModuleList([
            EvolveGCNLayer(channels) for _ in range(depth)
        ])
        self.activation = nn.SiLU()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = self.activation(x)
        return x

class EvolveGCNNeck(nn.Module):
    """EvolveGCN-based neck replacing PANet"""
    def __init__(self, backbone_channels=[256, 512, 1024]):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)

        # Top-down pathway
        self.top_conv = EvolveGCNBlock(backbone_channels[2])
        self.mid_conv = EvolveGCNBlock(backbone_channels[1])
        self.low_conv = EvolveGCNBlock(backbone_channels[0])

        # Bottom-up pathway
        self.up_conv1 = EvolveGCNBlock(backbone_channels[0])
        self.up_conv2 = EvolveGCNBlock(backbone_channels[1])

    def forward(self, features):
        c3, c4, c5 = features

        # Top-down pathway
        p5 = self.top_conv(c5)
        p4 = self.mid_conv(torch.cat([c4, self.upsample(p5)], dim=1))
        p3 = self.low_conv(torch.cat([c3, self.upsample(p4)], dim=1))

        # Bottom-up pathway
        n3 = self.up_conv1(p3)
        n4 = self.up_conv2(torch.cat([p4, self.downsample(n3)], dim=1))
        n5 = self.top_conv(torch.cat([p5, self.downsample(n4)], dim=1))

        return [n3, n4, n5]

class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()

    def replace_neck(self):
        # Get backbone output channels
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        
        # Replace neck with EvolveGCN
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  # Original PANet neck
                self.model.model[i] = EvolveGCNNeck(backbone_channels)
                break

# Training configuration remains similar
model = ModifiedYOLO('yolov8s.pt')

model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)



Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.48it/s]


                   all        175        632      0.509      0.455      0.425      0.221
                  Cone         52        203      0.666      0.675        0.7      0.362
           Face_Shield         25         29      0.599      0.138      0.348      0.156
                Gloves         40         87      0.748      0.306      0.407      0.232
               Goggles         31         37      0.647      0.348      0.404      0.193
                  Head          4         32      0.165      0.737        0.3      0.161
                Helmet         50        240      0.707      0.479       0.57      0.321
             No gloves          2          4     0.0324        0.5      0.248      0.124


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.8ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7988a889ab60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [11]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# JODIE

In [12]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv

# JODIE Model Implementation
class JODIE(nn.Module):
    """
    JODIE: Joint Dynamic Embedding Trajectory in Temporal Interaction Networks
    """
    def __init__(self, num_users, num_items, embedding_dim, hidden_dim):
        super(JODIE, self).__init__()
        
        # Embeddings for users and items
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)
        
        # RNN for updating embeddings
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        
        # Projection operator for predicting future embeddings
        self.projection = nn.Linear(hidden_dim + 1, embedding_dim)  # +1 for time difference
        
        # Parameters for initialization
        nn.init.xavier_uniform_(self.user_embeddings.weight)
        nn.init.xavier_uniform_(self.item_embeddings.weight)

    def forward(self, user_ids, item_ids, timestamps, prev_embeddings=None):
        """
        Forward pass for JODIE.
        
        Args:
            user_ids (torch.Tensor): Tensor of user IDs [batch_size]
            item_ids (torch.Tensor): Tensor of item IDs [batch_size]
            timestamps (torch.Tensor): Tensor of interaction timestamps [batch_size]
            prev_embeddings (torch.Tensor): Previous embeddings of users/items [batch_size, embedding_dim]
        
        Returns:
            updated_embeddings (torch.Tensor): Updated embeddings of users/items [batch_size, embedding_dim]
        """
        # Get initial embeddings for users and items
        user_embeds = self.user_embeddings(user_ids)
        item_embeds = self.item_embeddings(item_ids)
        
        # Compute interaction embedding
        interaction_embedding = user_embeds * item_embeds  # Element-wise product
        
        # If previous embeddings are not provided, initialize them
        if prev_embeddings is None:
            prev_embeddings = torch.zeros_like(interaction_embedding)
        
        # Prepare input for RNN: concatenate interaction embedding and previous embedding
        rnn_input = torch.cat([interaction_embedding.unsqueeze(1), prev_embeddings.unsqueeze(1)], dim=1)
        
        # Pass through RNN
        _, hidden_state = self.rnn(rnn_input)
        
        # Compute time difference (for simplicity, assume timestamps are normalized)
        time_diff = timestamps.unsqueeze(-1)  # Shape: [batch_size, 1]
        
        # Concatenate hidden state and time difference for projection
        projection_input = torch.cat([hidden_state.squeeze(0), time_diff], dim=1)
        
        # Project to get updated embeddings
        updated_embeddings = self.projection(projection_input)
        
        return updated_embeddings

# Custom Neck for YOLOv8 + JODIE
class JODIEYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml', num_users=1000, num_items=500, embedding_dim=64, hidden_dim=128):
        super().__init__(model_path)
        
        # Initialize JODIE
        self.jodie = JODIE(num_users=num_users, num_items=num_items, embedding_dim=embedding_dim, hidden_dim=hidden_dim)
        
        # Replace the neck with a custom module that integrates JODIE
        self.replace_neck()

    def replace_neck(self):
        """
        Replace the PANet neck in YOLOv8 with a custom neck that integrates JODIE.
        """
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  # Replace the neck
                self.model.model[i] = JODIEFusionNeck(backbone_channels=backbone_channels, jodie=self.jodie)
                break

# Custom Neck with JODIE Integration
class JODIEFusionNeck(nn.Module):
    """
    Custom neck for YOLOv8 that integrates JODIE for temporal modeling.
    """
    def __init__(self, backbone_channels=[256, 512, 1024], jodie=None):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        # Convolutions for channel alignment
        self.c5_conv = Conv(backbone_channels[2], backbone_channels[2], 1)
        self.c4_conv = Conv(backbone_channels[1], backbone_channels[1], 1)
        self.c3_conv = Conv(backbone_channels[0], backbone_channels[0], 1)
        
        # JODIE integration
        self.jodie = jodie

    def forward(self, features, user_ids, item_ids, timestamps):
        c3, c4, c5 = features
        
        # Process C5
        p5 = self.c5_conv(c5) + c5
        
        # Use JODIE to update embeddings based on temporal interactions
        updated_embeddings = self.jodie(user_ids, item_ids, timestamps)
        
        # Reshape updated embeddings to match feature map dimensions
        batch_size, channels, height, width = p5.shape
        updated_embeddings = updated_embeddings.view(batch_size, channels, 1, 1).expand(-1, -1, height, width)
        
        # Add updated embeddings to feature maps
        p5 = p5 + updated_embeddings
        p5_up = self.upsample(p5)
        
        # Process C4
        p4 = self.c4_conv(c4) + c4
        p4 = torch.cat([p4, p5_up], dim=1)
        p4_up = self.upsample(p4)
        
        # Process C3
        p3 = self.c3_conv(c3) + c3
        p3 = torch.cat([p3, p4_up], dim=1)
        
        # Bottom-up pathway
        p3_ds = self.downsample(p3)
        p4 = torch.cat([p4, p3_ds], dim=1)
        
        p4_ds = self.downsample(p4)
        p5 = torch.cat([p5, p4_ds], dim=1)
        
        return [p3, p4, p5]

# Training Configuration
num_users = 1000  # Number of unique users/objects in the dataset
num_items = 500   # Number of unique items/objects in the dataset
embedding_dim = 64  # Dimension of embeddings
hidden_dim = 128    # Hidden dimension of RNN

# Initialize the modified YOLOv8 model with JODIE
model = JODIEYOLO(
    model_path='yolov8s.pt',
    num_users=num_users,
    num_items=num_items,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim
)

# Example training configuration
model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)

Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.76it/s]


                   all        175        632      0.701      0.794      0.699      0.443
                  Cone         52        203      0.952      0.783      0.915      0.616
           Face_Shield         25         29      0.928      0.888      0.945      0.573
                Gloves         40         87      0.924      0.841      0.871      0.595
               Goggles         31         37      0.734       0.73      0.768      0.499
                  Head          4         32        0.4      0.875      0.404      0.245
                Helmet         50        240      0.917      0.694      0.811      0.498
             No gloves          2          4     0.0533       0.75      0.178     0.0743


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 12.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7988508d17b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [13]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# M2DNE

In [14]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00


In [15]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from torch_geometric.nn import GCNConv

# Define the M2DNE module
class M2DNE(nn.Module):
    def __init__(self, in_channels, out_channels, num_layers=2):
        super().__init__()
        self.convs = nn.ModuleList([GCNConv(in_channels, out_channels) for _ in range(num_layers)])
    
    def forward(self, x, edge_indices_list):
        for i, (conv, edge_index) in enumerate(zip(self.convs, edge_indices_list)):
            x = torch.relu(conv(x, edge_index))
        return x

# Define the M2DNE-based Neck
class M2DNENeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], num_layers=2):
        super().__init__()
        self.m2dne_blocks = nn.ModuleList([
            M2DNE(backbone_channels[2], backbone_channels[2], num_layers),
            M2DNE(backbone_channels[1], backbone_channels[1], num_layers),
            M2DNE(backbone_channels[0], backbone_channels[0], num_layers)
        ])

    def forward(self, features, edge_indices_list):
        c3, c4, c5 = features
        p5 = self.m2dne_blocks[0](c5, edge_indices_list)
        p4 = self.m2dne_blocks[1](c4, edge_indices_list)
        p3 = self.m2dne_blocks[2](c3, edge_indices_list)
        return [p3, p4, p5]

# Modified YOLO Class with M2DNE Neck
class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        # Extract backbone channels
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]

        # Replace the neck with M2DNENeck
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = M2DNENeck(backbone_channels=backbone_channels)
                break
    
    def forward(self, x, edge_indices_list):
        # Backbone
        features = self.model.model.backbone(x)  # Properly extract YOLOv8 backbone features

        # Neck (M2DNE)
        features = self.model.model.neck(features, edge_indices_list)  # Pass through M2DNE Neck
        
        # Head
        outputs = self.model.model.head(features)
        return outputs


# Training configuration
model = ModifiedYOLO('yolov8s.pt')

# Example usage
# You need to provide `edge_indices_list` (list of adjacency matrices) during training/inference
edge_indices_list = [torch.tensor([[0, 1], [1, 0]], dtype=torch.long)]  # Example adjacency matrix

model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)




Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.74it/s]


                   all        175        632      0.701      0.794      0.699      0.443
                  Cone         52        203      0.952      0.783      0.915      0.616
           Face_Shield         25         29      0.928      0.888      0.945      0.573
                Gloves         40         87      0.924      0.841      0.871      0.595
               Goggles         31         37      0.734       0.73      0.768      0.499
                  Head          4         32        0.4      0.875      0.404      0.245
                Helmet         50        240      0.917      0.694      0.811      0.498
             No gloves          2          4     0.0533       0.75      0.178     0.0743


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7988964e6f50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [16]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# MPNN

In [17]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

# Define the MPNN Layer
class MPNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPNNLayer, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node features.
        x = self.lin(x)

        # Step 3: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, out_channels]
        return x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        return aggr_out

# Custom MPNN Neck for YOLOv8
class MPNNNeck(nn.Module):
    """MPNN-based multi-scale fusion neck with residual connections"""
    def __init__(self, backbone_channels=[256, 512, 1024], hidden_dim=256):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        # Convolutions for channel alignment
        self.c5_conv = Conv(backbone_channels[2], backbone_channels[2], 1)
        self.c4_conv = Conv(backbone_channels[1], backbone_channels[1], 1)
        self.c3_conv = Conv(backbone_channels[0], backbone_channels[0], 1)
        
        # MPNN layers for each scale
        self.mpnn_layers = nn.ModuleList([
            MPNNLayer(backbone_channels[2], hidden_dim),
            MPNNLayer(backbone_channels[1], hidden_dim),
            MPNNLayer(backbone_channels[0], hidden_dim)
        ])
        
        # Final convolutions to match output channels
        self.final_convs = nn.ModuleList([
            Conv(hidden_dim, backbone_channels[2], 1),
            Conv(hidden_dim, backbone_channels[1], 1),
            Conv(hidden_dim, backbone_channels[0], 1)
        ])

    def forward(self, features):
        c3, c4, c5 = features
        
        # Process C5 with MPNN
        p5 = self.c5_conv(c5) + c5
        p5 = self.mpnn_process(p5, self.mpnn_layers[0])
        p5_up = self.upsample(p5)
        
        # Process C4 with MPNN
        p4 = self.c4_conv(c4) + c4
        p4 = torch.cat([p4, p5_up], dim=1)
        p4 = self.mpnn_process(p4, self.mpnn_layers[1])
        p4_up = self.upsample(p4)
        
        # Process C3 with MPNN
        p3 = self.c3_conv(c3) + c3
        p3 = torch.cat([p3, p4_up], dim=1)
        p3 = self.mpnn_process(p3, self.mpnn_layers[2])
        
        # Bottom-up pathway
        p3_ds = self.downsample(p3)
        p4 = torch.cat([p4, p3_ds], dim=1)
        p4 = self.mpnn_process(p4, self.mpnn_layers[1])
        
        p4_ds = self.downsample(p4)
        p5 = torch.cat([p5, p4_ds], dim=1)
        p5 = self.mpnn_process(p5, self.mpnn_layers[0])
        
        # Apply final convolutions to match output channels
        p3 = self.final_convs[2](p3)
        p4 = self.final_convs[1](p4)
        p5 = self.final_convs[0](p5)
        
        return [p3, p4, p5]

    def mpnn_process(self, feature_map, mpnn_layer):
        batch_size, channels, height, width = feature_map.shape
        
        # Flatten the feature map into nodes (spatial locations)
        x = feature_map.view(batch_size, channels, -1).permute(0, 2, 1)  # [B, H*W, C]
        
        # Create a fully connected graph (each node connects to every other node)
        num_nodes = height * width
        edge_index = torch.cartesian_prod(torch.arange(num_nodes), torch.arange(num_nodes)).t().contiguous()
        
        # Apply MPNN layer
        x = mpnn_layer(x, edge_index)
        
        # Reshape back to feature map
        x = x.permute(0, 2, 1).view(batch_size, -1, height, width)
        
        return x

# Modified YOLO class with MPNN Neck
class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  # Replace the neck
                self.model.model[i] = MPNNNeck(backbone_channels=backbone_channels)
                break

# Training configuration
model = ModifiedYOLO('yolov8s.pt')
model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)



Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.83it/s]


                   all        175        632      0.701      0.794      0.699      0.443
                  Cone         52        203      0.952      0.783      0.915      0.616
           Face_Shield         25         29      0.928      0.888      0.945      0.573
                Gloves         40         87      0.924      0.841      0.871      0.595
               Goggles         31         37      0.734       0.73      0.768      0.499
                  Head          4         32        0.4      0.875      0.404      0.245
                Helmet         50        240      0.917      0.694      0.811      0.498
             No gloves          2          4     0.0533       0.75      0.178     0.0743


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 13.4ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79889606b580>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [18]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# Node2Vec

In [19]:
!pip install node2vec

In [20]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv
import networkx as nx
from node2vec import Node2Vec
import numpy as np

class FeatureGraphBuilder(nn.Module):
    """Builds adaptive graphs from feature maps"""
    def __init__(self, k_neighbors=8, similarity_threshold=0.5):
        super().__init__()
        self.k = k_neighbors
        self.threshold = similarity_threshold
        
    def forward(self, x):
        """Input: [B, C, H, W] / Output: List of Graphs"""
        B, C, H, W = x.shape
        graphs = []
        
        # Create coordinate grid
        grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W))
        grid = torch.stack((grid_x, grid_y), dim=-1).float().to(x.device)
        
        for b in range(B):
            # Combine spatial and feature information
            features = x[b].permute(1,2,0)  # [H, W, C]
            combined = torch.cat([grid, features], dim=-1)  # [H, W, C+2]
            
            # Flatten to nodes
            nodes = combined.view(H*W, -1)
            
            # Build similarity graph
            G = nx.Graph()
            similarities = torch.cdist(nodes, nodes)
            
            for i in range(H*W):
                # Find top-k similar nodes
                _, indices = torch.topk(similarities[i], self.k+1, largest=False)
                for j in indices[1:]:  # Skip self
                    if similarities[i,j] < self.threshold:
                        G.add_edge(i, j.item())
            
            graphs.append(G)
        
        return graphs

class AdaptiveNode2Vec(nn.Module):
    """On-the-fly Node2Vec processing with feature alignment"""
    def __init__(self, channels, embed_dim=64):
        super().__init__()
        self.graph_builder = FeatureGraphBuilder()
        self.embed_dim = embed_dim
        self.proj = nn.Linear(embed_dim, channels)
        
    def forward(self, x):
        B, C, H, W = x.shape
        graphs = self.graph_builder(x)
        
        # Process each item in batch
        embeddings = []
        for b, G in enumerate(graphs):
            # Generate walks
            node2vec = Node2Vec(G, dimensions=self.embed_dim, num_walks=20, walk_length=15)
            model = node2vec.fit()
            
            # Get embeddings with spatial ordering
            emb_matrix = np.zeros((H*W, self.embed_dim))
            for node in G.nodes():
                emb_matrix[node] = model.wv[str(node)]
            
            # Project and reshape
            emb_tensor = self.proj(torch.tensor(emb_matrix, device=x.device).float())
            emb_tensor = emb_tensor.view(H, W, -1).permute(2,0,1)  # [C, H, W]
            embeddings.append(emb_tensor)
        
        return torch.stack(embeddings)

class Node2VecEnhancedNeck(nn.Module):
    """Multi-scale graph-enhanced neck"""
    def __init__(self, backbone_channels):
        super().__init__()
        # Original neck components
        self.pan = DyCAFNeck(backbone_channels)
        
        # Graph processors
        self.graph_processors = nn.ModuleList([
            AdaptiveNode2Vec(c) for c in backbone_channels
        ])
        
        # Fusion convolutions
        self.fusion_convs = nn.ModuleList([
            Conv(c*2, c, k=3, act='SiLU') for c in backbone_channels
        ])
        
    def forward(self, features):
        # Original neck processing
        pan_features = self.pan(features)
        
        # Graph-enhanced features
        graph_features = [proc(f) for proc, f in zip(self.graph_processors, features)]
        
        # Multi-scale fusion
        enhanced_features = []
        for p_f, g_f, conv in zip(pan_features, graph_features, self.fusion_convs):
            combined = torch.cat([p_f, g_f], dim=1)
            enhanced = conv(combined)
            enhanced_features.append(enhanced)
            
        return enhanced_features

class ModifiedYOLO(YOLO):
    def replace_neck(self):
        backbone_channels = [256, 512, 1024]  # Verify actual channels
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):
                self.model.model[i] = Node2VecEnhancedNeck(backbone_channels)
                break

# Usage
model = ModifiedYOLO('yolov8s.pt')
model.train(
    data=location,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5,
    patience=5 
)



Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.76it/s]


                   all        175        632      0.701      0.794      0.699      0.443
                  Cone         52        203      0.952      0.783      0.915      0.616
           Face_Shield         25         29      0.928      0.888      0.945      0.573
                Gloves         40         87      0.924      0.841      0.871      0.595
               Goggles         31         37      0.734       0.73      0.768      0.499
                  Head          4         32        0.4      0.875      0.404      0.245
                Helmet         50        240      0.917      0.694      0.811      0.498
             No gloves          2          4     0.0533       0.75      0.178     0.0743


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 12.3ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x798838497190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [21]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# SpikeNet

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO

# SpikeNet Implementation
class SpikeConv(nn.Module):
    """Spiking Convolutional Layer with Membrane Potential"""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dropout=0.5):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.dropout = nn.Dropout2d(dropout)
        self.membrane = nn.Parameter(torch.zeros(1, out_channels, 1, 1))
        self.threshold = 1.0
        
    def forward(self, x):
        mem_potential = self.membrane.repeat(x.size(0), 1, x.size(2), x.size(3))
        conv_out = self.conv(x)
        mem_potential = mem_potential + conv_out
        spike = (mem_potential >= self.threshold).float()
        mem_potential = mem_potential * (1 - spike)
        return self.dropout(spike)

class SpikeNet(nn.Module):
    """SpikeNet Neck Architecture"""
    def __init__(self, in_channels=[256, 512, 1024], out_channels=[256, 512, 1024], dropout=0.5):
        super().__init__()
        
        # Bottom-up pathway
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        # Lateral connections
        self.lateral_conv1 = SpikeConv(in_channels[2], out_channels[2], 1, dropout=dropout)
        self.lateral_conv2 = SpikeConv(in_channels[1], out_channels[1], 1, dropout=dropout)
        
        # Fusion layers
        self.fusion_conv1 = SpikeConv(out_channels[2], out_channels[2], 3, padding=1, dropout=dropout)
        self.fusion_conv2 = SpikeConv(out_channels[1], out_channels[1], 3, padding=1, dropout=dropout)
        self.fusion_conv3 = SpikeConv(out_channels[0], out_channels[0], 3, padding=1, dropout=dropout)

    def forward(self, features):
        c3, c4, c5 = features
    
        # Top-down pathway
        p5 = self.lateral_conv1(c5)
        p5_up = F.interpolate(p5, scale_factor=2, mode='nearest')
        p4 = self.lateral_conv2(c4) + self.adjust_conv(p5_up)  # Adjust channels
    
        p4_up = F.interpolate(p4, size=c3.shape[2:], mode='nearest')  # Match c3's spatial size
        p3 = c3 + p4_up  # No downsampling of c3
    
        # Bottom-up pathway
        n3 = self.fusion_conv1(p3)
        n3_up = F.interpolate(n3, scale_factor=0.5, mode='nearest')  # Downsample
        n4 = self.fusion_conv2(p4) + n3_up
    
        n4_up = F.interpolate(n4, scale_factor=0.5, mode='nearest')
        n5 = self.fusion_conv3(p5) + n4_up
    
        return [n3, n4, n5]

# YOLOv8 + SpikeNet Integration
class YOLOv8SpikeNet(YOLO):
    """Custom YOLOv8 with SpikeNet Neck"""
    def __init__(self, model='yolov8n.yaml', dropout=0.5):
        super().__init__(model)
        self.replace_neck(dropout)
    
    def replace_neck(self, dropout):
        # Replace PANet with SpikeNet in model architecture
        self.model.model[-1] = SpikeNet(
            in_channels=[256, 512, 1024],
            out_channels=[256, 512, 1024],
            dropout=dropout
        )

# Training Configuration
if __name__ == '__main__':
    # Initialize custom YOLOv8 + SpikeNet model
    model = YOLOv8SpikeNet(model='yolov8n.yaml')
    
    # Train the model
    results = model.train(
        data=location,
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="SGD",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.5,
    patience=5 
    )



Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train9/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train9
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 0.112 hours.
Optimizer stripped from runs/detect/train9/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train9/weights/best.pt, 6.2MB

Validating runs/detect/train9/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.96it/s]


                   all        175        632      0.652      0.518      0.507      0.284
                  Cone         52        203      0.889      0.632      0.777      0.403
           Face_Shield         25         29      0.774      0.621       0.67      0.429
                Gloves         40         87      0.817      0.586      0.679      0.392
               Goggles         31         37      0.757      0.337      0.501      0.285
                  Head          4         32        0.3      0.594      0.291      0.152
                Helmet         50        240      0.924      0.353      0.557      0.299
             No gloves          2          4      0.101        0.5      0.073     0.0299


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 8.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train9


In [23]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors

# TGAT

In [24]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f

# Revised TGATLayer with Static Image Support
class TGATLayer(nn.Module):
    def __init__(self, channels, num_heads=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(channels, num_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(channels, channels * 4), 
            nn.GELU(),
            nn.Linear(channels * 4, channels)
        )
        self.norm1 = nn.LayerNorm(channels)
        self.norm2 = nn.LayerNorm(channels)
        
    def forward(self, x):
        B, C, H, W = x.shape
        x_flat = x.view(B, C, -1).permute(0, 2, 1)  # [B, N, C]
        
        # Self-attention
        attn_out, _ = self.attention(x_flat, x_flat, x_flat)
        x_flat = self.norm1(x_flat + attn_out)
        
        # FFN
        ffn_out = self.ffn(x_flat)
        x_flat = self.norm2(x_flat + ffn_out)
        
        return x_flat.permute(0, 2, 1).view(B, C, H, W)

# Revised TGATNeck with Channel Management
class TGATNeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], num_heads=4):
        super().__init__()
        
        # Top-down path
        self.lateral_conv5 = Conv(backbone_channels[2], backbone_channels[1], 1)
        self.lateral_conv4 = Conv(backbone_channels[1], backbone_channels[0], 1)
        
        # TGAT blocks
        self.tgat5 = TGATLayer(backbone_channels[1], num_heads)
        self.tgat4 = TGATLayer(backbone_channels[0], num_heads)
        self.tgat3 = TGATLayer(backbone_channels[0], num_heads)
        
        # Bottom-up path
        self.up_conv3 = Conv(backbone_channels[0], backbone_channels[1], 3)
        self.up_conv4 = Conv(backbone_channels[1], backbone_channels[2], 3)

    def forward(self, features):
        c3, c4, c5 = features
        
        # Top-down
        p5 = self.lateral_conv5(c5)
        p4 = self.lateral_conv4(c4 + F.interpolate(p5, scale_factor=2))
        p3 = self.tgat3(c3 + F.interpolate(p4, scale_factor=2))
        
        # Bottom-up
        n3 = p3
        n4 = self.tgat4(p4 + self.up_conv3(F.max_pool2d(n3, 2)))
        n5 = self.tgat5(p5 + self.up_conv4(F.max_pool2d(n4, 2)))
        
        return [n3, n4, n5]

# Robust YOLOv8 Integration
class TGATYOLO(YOLO):
    def __init__(self, cfg='yolov8s.yaml', num_heads=4):
        super().__init__(cfg)
        self.replace_neck(num_heads)
        
    def replace_neck(self, num_heads):
        # Get actual backbone channels from C2f modules
        backbone_channels = [
            self.model.model[i].cv2.conv.out_channels  # Correct channel access
            for i in [4, 6, 8]  # These indices work for YOLOv8s v8.0.xx
        ]
        
        # Replace neck module (index -2 for most YOLOv8 versions)
        self.model.model[-2] = TGATNeck(backbone_channels, num_heads)

# Training Configuration
if __name__ == '__main__':
    model = TGATYOLO('yolov8s.yaml')
    model.train(
        data=location,
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="SGD",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.1  # Match dropout from TGATLayer
    )

Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/street-work-4/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /kaggle/working/street-work-4/train/labels.cache... 611 images, 2 backgrounds, 0 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/street-work-4/valid/labels.cache... 175 images, 19 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train10/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train10
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 0.183 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 22.5MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.70it/s]


                   all        175        632       0.58      0.666      0.599      0.333
                  Cone         52        203      0.771      0.778      0.833      0.434
           Face_Shield         25         29      0.625      0.748       0.79      0.469
                Gloves         40         87      0.805      0.667      0.774      0.439
               Goggles         31         37        0.7      0.676      0.705      0.405
                  Head          4         32      0.323       0.75      0.321      0.178
                Helmet         50        240      0.787      0.542      0.647      0.342
             No gloves          2          4     0.0453        0.5      0.124     0.0619


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.7ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train10


In [25]:
import gc
gc.collect()

import torch
torch.cuda.empty_cache()  # Frees unused cached memory
torch.cuda.ipc_collect()  # Collects unreferenced tensors